In [ ]:
from pyspark.sql.types import StructType,StringType,DoubleType,StructField
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassificationModel

spark = SparkSession.builder.appName("Log Watcher").getOrCreate()

logSchema = StructType([StructField("sl",DoubleType(),True),
                        StructField("sw",DoubleType(),True),
                        StructField("pl",DoubleType(),True),
                       StructField("pw",DoubleType(),True),
                       StructField("class",StringType(),True)])
irisDF = spark.readStream.\
        option("sep", ",").\
        schema(logSchema).csv("logs")


vec = VectorAssembler(inputCols=irisDF.columns[0:4],outputCol="features")
irisDF = vec.transform(irisDF)
model = DecisionTreeClassificationModel.load("dtModel")
irisDF = model.transform(irisDF)

query = irisDF.writeStream\
            .outputMode("update")\
            .format("console").start()

query.awaitTermination()